In [270]:
import pandas as pd
import datetime

In [271]:
root_dir='/home/jovyan/labs/SlaterFireHydrology';
data_dir=root_dir+'/data';
processed_dir =data_dir+'/Processed';
raw_dir=data_dir+'/Raw';

In [272]:
precip_data_fn=raw_dir+'/salmonrivercavejunct.csv'
output_file=processed_dir+'/PrecipFlowData.txt'

In [273]:
#pd.read_csv(precip_data_fn).tail()

,STATION,NAME,DATE,DAPR,MDPR,PRCP
4292,US1ORJS0024,"CAVE JUNCTION 0.4 ENE, OR US",2022-03-03,NaN,NaN,4.1
4293,US1ORJS0024,"CAVE JUNCTION 0.4 ENE, OR US",2022-03-04,NaN,NaN,0.0
4294,US1ORJS0024,"CAVE JUNCTION 0.4 ENE, OR US",2022-03-05,NaN,NaN,5.1
4295,US1ORJS0024,"CAVE JUNCTION 0.4 ENE, OR US",2022-03-06,NaN,NaN,0.0
4296,US1ORJS0024,"CAVE JUNCTION 0.4 ENE, OR US",2022-03-07,NaN,NaN,0.0


In [274]:
weather_stations=['happycamp', 'orleans','salmonriver','cavejunction']
precip_data_fns={'happycamp': '/happycampprecipdata.csv',\
                 'orleans': '/orleansprecipdata.csv',\
                 'salmonriver':'/salmonrivercavejunct.csv',\
                 'cavejunction':'/salmonrivercavejunct.csv'}
station_name = {'happycamp': 'USC00043761',\
                 'orleans': 'USC00046508',\
                 'salmonriver':'US1CASK0009',\
                 'cavejunction':'US1ORJS0024'}

In [275]:
precip_df = pd.read_csv(raw_dir+precip_data_fns[weather_stations[0]])[['DATE', 'PRCP', 'TMAX','TMIN','TOBS']].set_index('DATE')
precip_df=precip_df.rename(columns={'PRCP':'PRCP_'+weather_stations[0], 'TMAX':'TMAX_'+weather_stations[0], \
                          'TMIN':'TMIN_'+weather_stations[0], 'TOBS':'TOBS_'+weather_stations[0]})

In [276]:
for station in weather_stations[1:]:
    precip_station_df = pd.read_csv(precip_data_fn)[['STATION','DATE', 'PRCP']].set_index('DATE')
    precip_station_df=precip_station_df[precip_station_df['STATION']==station_name[station]]
    precip_station_df=precip_station_df.drop(columns=['STATION'])
    precip_station_df=precip_station_df.rename(columns={'PRCP':'PRCP_' + station})
    precip_df=precip_df.join(precip_station_df, how='outer')


In [277]:
precip_df.head()

,PRCP_happycamp,TMAX_happycamp,TMIN_happycamp,TOBS_happycamp,PRCP_orleans,PRCP_salmonriver,PRCP_cavejunction
DATE,,,,,,,
2015-01-01,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2015-01-02,0.0,48.0,17.0,18.0,NaN,0.0,NaN
2015-01-03,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2015-01-04,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2015-01-05,0.0,NaN,NaN,33.0,NaN,0.0,NaN


In [278]:
river_names=['IndianCreek','SalmonRiver','SuckerCreek']
discharge_map={'IndianCreek': '16574_00060', 'SalmonRiver':'47824_00060','SuckerCreek':'117619_00060'}
gage_ht_map={'IndianCreek': '16575_00065','SalmonRiver':'47825_00065','SuckerCreek':'117620_00065'}
header_line_num={'IndianCreek': 30, 'SalmonRiver':31,'SuckerCreek':29}

In [279]:

begin_date = '2015-01-01'
end_date='2022-03-10'

output_df = pd.DataFrame({'DATE':pd.date_range(begin_date, end_date)}).set_index('DATE')
output_df=output_df.join(precip_df.set_index(pd.DatetimeIndex(precip_df.index)), how='outer')

In [280]:
output_df.head()

,PRCP_happycamp,TMAX_happycamp,TMIN_happycamp,TOBS_happycamp,PRCP_orleans,PRCP_salmonriver,PRCP_cavejunction
DATE,,,,,,,
2015-01-01,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2015-01-02,0.0,48.0,17.0,18.0,NaN,0.0,NaN
2015-01-03,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2015-01-04,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2015-01-05,0.0,NaN,NaN,33.0,NaN,0.0,NaN


In [281]:
output_df=precip_df
for river in river_names:
    flow_filename=raw_dir+'/'+ river+'FlowData.txt'
    dataframe=pd.read_csv(flow_filename,header=header_line_num[river], skiprows=[header_line_num[river]+1],\
                      delimiter='\t', low_memory=False)
    dataframe.rename(mapper={discharge_map[river]:'discharge_'+river,gage_ht_map[river]:'gage_ht_'+river},inplace=True, axis="columns")
    flow_df = dataframe[['datetime','discharge_'+river,'gage_ht_'+river]]
    flow_df['DATE']=[str[:-6] for str in flow_df['datetime']]
    flow_df.drop(columns=['datetime'])
    flow_agg_df = flow_df.groupby(by='DATE').agg('mean')
    flow_agg_df.columns = flow_agg_df.columns.get_level_values(0)
    flow_agg_df['discharge_'+river+'_daily_change']=flow_agg_df['discharge_'+river].diff(periods=1) #Diff from previous day
    output_df = output_df.join(flow_agg_df, how='outer')
    

/tmp/ipykernel_80/1879786212.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow_df['DATE']=[str[:-6] for str in flow_df['datetime']]


In [282]:
output_df.head()

,PRCP_happycamp,TMAX_happycamp,TMIN_happycamp,TOBS_happycamp,PRCP_orleans,PRCP_salmonriver,PRCP_cavejunction,discharge_IndianCreek,gage_ht_IndianCreek,discharge_IndianCreek_daily_change,discharge_SalmonRiver,gage_ht_SalmonRiver,discharge_SalmonRiver_daily_change,discharge_SuckerCreek,gage_ht_SuckerCreek,discharge_SuckerCreek_daily_change
DATE,,,,,,,,,,,,,,,,
2015-01-01,NaN,NaN,NaN,NaN,NaN,0.0,NaN,310.656250,4.981979,NaN,NaN,NaN,NaN,167.927083,1.787083,NaN
2015-01-02,0.0,48.0,17.0,18.0,NaN,0.0,NaN,290.437500,4.922396,-20.218750,NaN,NaN,NaN,157.031250,1.755104,-10.895833
2015-01-03,NaN,NaN,NaN,NaN,NaN,0.0,NaN,272.614583,4.867604,-17.822917,NaN,NaN,NaN,145.177083,1.719583,-11.854167
2015-01-04,NaN,NaN,NaN,NaN,NaN,0.0,NaN,257.375000,4.817917,-15.239583,NaN,NaN,NaN,135.562500,1.688542,-9.614583
2015-01-05,0.0,NaN,NaN,33.0,NaN,0.0,NaN,244.250000,4.774167,-13.125000,NaN,NaN,NaN,129.333333,1.668854,-6.229167


In [283]:
split_date=output_df.index.str.split('-', expand=True)

In [284]:
output_df['year']=split_date.get_level_values(0)
output_df['month']=split_date.get_level_values(1)
output_df['day']=split_date.get_level_values(2)

In [285]:
output_df.to_csv(output_file)